In [1]:
!pip install bert-score

In [12]:
import json
import numpy as np

from bert_score import score

In [13]:
file_name = 'base_paligemma_uamemes_predicted_answers (2).json'

with open(file_name, 'r') as file:
    results = json.load(file)

In [14]:
ground_truth = list(map(lambda entry: entry['ground-truth-answer'], results))
predicted = list(map(lambda entry: entry['answer'], results))

In [15]:
precisions, recalls, f1_scores = score(predicted, ground_truth, lang="uk", verbose=True)

calculating scores...
computing bert embedding.


  0%|          | 0/8 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/5 [00:00<?, ?it/s]

done in 3.48 seconds, 83.59 sentences/sec


In [16]:
precisions = list(map(lambda score: score.item(), precisions))
recalls = list(map(lambda score: score.item(), recalls))
f1_scores = list(map(lambda score: score.item(), f1_scores))

In [17]:
for i, (precision, recall, f1) in enumerate(zip(precisions, recalls, f1_scores)):
    metric = {
        'precision': precision,
        'recall': recall,
        'f1': f1
    }
    results[i]['metric'] = metric

In [18]:
print(f"Average Precision: {np.mean(precisions)}")
print(f"Average Recall: {np.mean(recalls)}")
print(f"Average F1: {np.mean(f1_scores)}")

Average Precision: 0.6300064021164609
Average Recall: 0.6444388361730936
Average F1: 0.6357451733649802


In [19]:
top_10_indices = np.argsort(f1_scores)[-10:][::-1]
np.array(results)[top_10_indices]

array([{'question': 'Про яку подію йдеться у цьому мемі?', 'ground-truth-answer': 'Вибух на Кримському мосту', 'answer': 'Вибух на мосту Арапучи.', 'metric': {'precision': 0.8338742256164551, 'recall': 0.8542201519012451, 'f1': 0.8439245820045471}},
       {'question': 'Про які країни чи міста йдеться в цьому мемі?', 'ground-truth-answer': 'Москва', 'answer': 'Росія та Москва.', 'metric': {'precision': 0.7679848670959473, 'recall': 0.8300695419311523, 'f1': 0.7978212237358093}},
       {'question': 'Про яку подію йдеться у цьому мемі?', 'ground-truth-answer': 'Відступ російської армії з Харківщини', 'answer': 'Винищування російських військ у Вікторії.', 'metric': {'precision': 0.7873832583427429, 'recall': 0.8038998246192932, 'f1': 0.795555830001831}},
       {'question': 'Про яку подію йдеться у цьому мемі?', 'ground-truth-answer': 'Виявилося, що російські військові копали окопи в Чорнобильській зоні', 'answer': 'Російські військові входить у Чернобиль.', 'metric': {'precision': 0.835

In [20]:
worst_10_indices = np.argsort(f1_scores)[:10]
np.array(results)[worst_10_indices]

array([{'question': 'Про які країни чи міста йдеться в цьому мемі?', 'ground-truth-answer': 'Київська обл;Київ', 'answer': 'Кіт на зображенні зображений у кількох країнах та містах, включаючи Україну, Україну, Україну, Україну, Україну, Україну, Україну, Україну, Україну, Україну, Україну, Україну, Україну, Україну, Україну, Україну, Україну, Україну, Україну, Україну, Україну, Україну, Україну, Україну, Україну, Україну', 'metric': {'precision': 0.44431763887405396, 'recall': 0.5617101192474365, 'f1': 0.4961647093296051}},
       {'question': 'Про яку подію йдеться у цьому мемі?', 'ground-truth-answer': 'У Москві пройшов традиційний парад Перемоги. Був лише один танк.', 'answer': 'Парад.', 'metric': {'precision': 0.5168226957321167, 'recall': 0.48101097345352173, 'f1': 0.49827420711517334}},
       {'question': 'Про які країни чи міста йдеться в цьому мемі?', 'ground-truth-answer': 'Німеччина', 'answer': 'УКРАЇНА.', 'metric': {'precision': 0.47847452759742737, 'recall': 0.556719601154

In [21]:
result_filename = file_name.split('.')[0] + 'with_metrics.' + file_name.split('.')[1]

with open(result_filename, "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=4)